In [145]:
from selenium import webdriver
import pandas as pd 
import pymongo as mg
import time

## 웹사이트 : 하이닥 
- https://www.hidoc.co.kr/

In [143]:
browser = webdriver.Chrome(executable_path='../chromedriver.exe') 
browser.get('https://www.hidoc.co.kr/integratesearch/searchhealthqnalist?query=%EC%B2%99%EC%B6%94')

In [3]:
browser.implicitly_wait(10)

### 커뮤니티 정보 
- 전체 : #hidocBody > div > div.coll_cont.coll_qna > div.coll_info > ul > li:nth-child(1)
- 제목 :  div.coll_info > ul > li:nth-child(1) > a  /link_qna
- 내용 : #hidocBody > div > div.coll_cont.coll_qna > div.coll_info > ul > li:nth-child(1) > p.desc_txt
- 답변 : #hidocBody > div > div.coll_cont.coll_qna > div.coll_info > ul > li:nth-child(1) > p.desc_answer
- 

### 해당 내용 
- 제목 : div.box_type1.view_question > div.inner > strong
- 내용 : div.desc > p
- 게시 날짜 : div.inner > div.clear_g > div.user_info > span
- 태크 : #hidocBody > div.health_view > div.box_type1.view_question > div.inner > div.txt_tag

- 답변제목 : #hidocBody > div.health_view > div.view_answer > div:nth-child(28) > strong
- 답변자 : div:nth-child(29) > div.answer_head > div.doctor_clinic > a
- 답변자 소속 : div:nth-child(29) > div.answer_head > div.doctor_clinic > span
- 답변 내용 : #hidocBody > div.health_view > div.view_answer > div:nth-child(29) > div.answer_body > div > div:nth-child(1)

In [4]:
# 한 게시글 클릭 
click_comm= browser.find_element_by_css_selector('div.coll_info > ul > li:nth-child(1) > a').click()

In [128]:
for next_page in range(1,10) :
    # next_page = 'span > a.ico_comm.btn_next'
    for page_number in range(2, 11) : # 첫 페이지는 tag가 달라짐
        element_path = '#hidocBody > div > div.coll_cont.coll_qna > div.paging_type1 > span > a:nth-child({})'.format(page_number)
        try : 
            pagination = browser.find_element_by_css_selector(element_path)
            pagination.click()
            time.sleep(2) # term을 주기
            print("성공 : {} ".format(page_number))
        except : 
            print("오류")
    next_page = browser.find_element_by_css_selector('span > a.ico_comm.btn_next').click()
    

성공 : 2 
성공 : 3 
성공 : 4 
성공 : 5 
성공 : 6 
성공 : 7 
성공 : 8 
성공 : 9 
성공 : 10 
오류
성공 : 3 
성공 : 4 
성공 : 5 
성공 : 6 
성공 : 7 
성공 : 8 
성공 : 9 
성공 : 10 
오류
성공 : 3 
성공 : 4 
오류
오류
오류
오류
오류
오류


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.180)


In [142]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['data_spine_hidoc']

comm_list = []
for page_number in range(1, 10) : # 1에서 10까지 페이지 순회

    for pagenation in range(2,11):
        page_path = '#hidocBody > div > div.coll_cont.coll_qna > div.paging_type1 > span > a:nth-child({})'.format(pagenation)
        try : 
            time.sleep(1) # term을 주기
            print("성공 : {} ".format(pagenation))
            
            # 질문 리스트 가져오기 
            questions_bundle = browser.find_elements_by_css_selector('.link_qna')

            # 게시물 질문 가져오기 
            for question_bundle in questions_bundle:
                # 각 질문 클릭
                question_bundle.click()

                # 질문에 관한 정보 
                try:
                    question_title = browser.find_element_by_css_selector('div.box_type1.view_question > div.inner > strong').text
                except : 
                    question_title = str()
                try:
                    question_content = browser.find_element_by_css_selector('div.desc > p').text
                except :
                    question_content = str()
                try:
                    question_date = browser.find_element_by_css_selector('div.inner > div.clear_g > div.user_info > span').text
                except :
                    question_date = str()
                try : 
                    question_tag = browser.find_element_by_css_selector('div.inner > div.txt_tag').text
                except :
                    question_tag = str()

                # 답변의 총 갯수 파악
                answers_bundle = browser.find_elements_by_css_selector('.box_type1.hidoc_answer')
                len(answers_bundle)

                answer_list= []
                for answer in answers_bundle: 
                    try : 
                        answer_title = browser.find_element_by_css_selector('div.view_answer > div:nth-child(28) > strong').text
                    except :
                        answer_title = str()
                    try : 
                        answer_name = browser.find_element_by_css_selector('div:nth-child(29) > div.answer_head > div.doctor_clinic > a').text
                    except: 
                        answer_name = str()
                    try : 
                        answer_belong = browser.find_element_by_css_selector('div:nth-child(29) > div.answer_head > div.doctor_clinic > span').text
                    except:
                        answer_belong=str()
                    try : 
                        answer_content = browser.find_element_by_css_selector('div.view_answer > div:nth-child(29) > div.answer_body > div > div:nth-child(1)').text
                    except:
                        answer_content = str()

                    answers_list = [question_title, question_content, question_date, question_tag, answer_title, answer_name, answer_belong, answer_content ]
                    answer_list.append(answers_list)
                browser.back() #팝업 닫기
                
            pagination = browser.find_element_by_css_selector(page_path)
            pagination.click()
            
            # 현재 페이지의 데이터를 comm_list에 추가
            comm_list.extend(answer_list)
            # answer_list 초기화
            answer_list = []
            
        except : 
            print("오류")
            
    next_page = browser.find_element_by_css_selector('span > a.ico_comm.btn_next').click()
        
    # 저장을 위한 작업
    comm_colums_name = ['question_title', 'question_content', 'question_date', 'question_tag', 'answer_title', 'answer_name', 'answer_belong', 'answer_content' ]
    df_comm = pd.DataFrame(data=comm_list, columns=comm_colums_name)
    data_dict = df_comm.to_dict(orient='records')
    collection.insert_many(data_dict) 
        

성공 : 2 
성공 : 3 
성공 : 4 
성공 : 5 
오류
성공 : 6 
오류
성공 : 7 
오류
성공 : 8 
오류
성공 : 9 
오류
성공 : 10 
오류


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.180)
